# RTML Lab 11: GNNs

Today we explore GNNs, in particular [Graph Convolutional Networks by Kipf and Welling (2017)](https://arxiv.org/abs/1609.02907).

This tutorial is based on the [PyTorch Geometric tutorial for GCN](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html).

## Setup

PyG only seems to work with recent versions of PyTorch, so you may need to upgrade:

In [1]:
id = 123012
name = 'Todsavad Tangtortan'

In [1]:
# !pip install --upgrade torch
# !pip install torch_geometric

Be sure to restart your kernel if you're doing this in Jupyter.

## Our first graph

OK, with that set up, let's use PyG to build a graph:

In [1]:
import torch_geometric
torch_geometric.__version__

'2.3.0'

In [2]:
import torch
from torch_geometric.data import Data

# os.environ['http_proxy'] = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
    
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [3]:
data

Data(x=[3, 1], edge_index=[2, 4])

So we see that this defines a graph with three nodes and two edges. The edge index
is in so-called [COO format](https://pytorch.org/docs/stable/sparse.html#sparse-coo-docs) (a format for representing sparse matrices)
and in this case is a $2\times 2|E|$ tensor.

The indices are indexing the set of nodes, in the range 0 to $N-1$.

The following is equivalent. Note that each undirected edge is denoted by two pairs of indices:

In [4]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

The tensor `x` is providing the *features* of the nodes in the graph, in this case a scalar for each node. As we know,
node features can be vectors as well, but all nodes must have the same feature dimensionality.

We can check the validity of our graph with the `validate()` method:

In [5]:
data.validate(raise_on_error=True)

True

Try some other attributes and:
- `x`
- `edge_index`
- `edge_attr`
- `num_nodes`
- `num_edges`
- `num_node_features`
- `has_isolated_nodes()`
- `has_self_loops()`
- `is_directed()`

And you can move a graph to a GPU just like a tensor:

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

## PyG datasets (Cora)

Let's load the Cora dataset:

In [7]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./data/Cora', name='Cora')

In [8]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

Try a few of the dataset's methods:
- `len(dataset)`
- `dataset.num_classes`
- `dataset.num_node_features`
- `dataset[0]`

From this we see that Cora is a single graph.
We will only be doing transduction with this dataset.
After assigning the single graph to a variable such as `data`,
explore its characteristics:
- `data.is_undirected()`
- `data.train_mask.sum().item()`
- `data.val_mask.sum().item()`
- `data.test_mask.sum().item()`

As this graph is mainly used to test semi-supervised learning algorithms, we see that the validation and test data are more numerous than the training data.


## Mini-batches

How can we train on multiple examples in parallel? PyG puts multiple graphs together with block-diagonal adjacency matrices and concatenated features.
See the [tutorial section on mini-batches](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#mini-batches) for more
information.

For a GCN on Cora, however, we don't need fancy data loaders or parallel mini-batches, as we're dealing with just one graph.

## Define a GCN

So let's define a GCN for Cora, then:

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

Moving the model to a GPU and training is the same as for any other PyTorch model. The "data" is treated as a single example:

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0: 
        print(f'Epoch : {epoch+1}')

Epoch : 50
Epoch : 100
Epoch : 150
Epoch : 200


After training, we an evaluate on the test dataset:

In [11]:
model.eval() #No dropout
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7970


## Exercises

1. Implement early stopping using the validation set for the Cora GCN.
2. Reproduce the rest of the results from Kipf and Welling with your GCN.
3. Implement the [Graph Attention Network](https://arxiv.org/abs/1710.10903) and reproduce their results on the datasets available via PyG.


## 1. Early Stopping 
Implement early stopping using the validation set for the Cora GCN.

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
num_epochs = 200

In [197]:
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def valid(data):
    model.eval() #No dropout
    out = model(data)
    loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
    loss.backward()
    return loss

def test(data):
    model.eval() #No dropout
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

def early_stopping(train_loss, validation_loss, min_delta, tolerance):
    counter = 0
    if (validation_loss - train_loss) > min_delta:
        counter +=1
        if counter >= tolerance:
          return True

In [198]:
import numpy as np
# best_val_loss = final_acc = 0 
val_loss_old = np.infty
for epoch in range(num_epochs):
    train_loss = train(data)
    val_loss  = valid(data)
    test_acc = test(data)
    # early stopping
    if early_stopping(train_loss, val_loss, min_delta=1, tolerance = 10):
      print("We are at epoch:", epoch)
      break

    if (epoch+1) % 50 == 0: 
        print(f'Epoch : {epoch+1:03d}, Loss: {val_loss:.4f}, Test Acc: {test_acc:.4f}')

Epoch : 050, Loss: 0.7416, Test Acc: 0.7990
Epoch : 100, Loss: 0.7499, Test Acc: 0.8110
Epoch : 150, Loss: 0.7670, Test Acc: 0.8090
Epoch : 200, Loss: 0.7974, Test Acc: 0.8030


## 2. Kipf and Welling
Reproduce the rest of the results from Kipf and Welling with your GCN.

In [127]:
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import NELL

dataset_Citeseer = Planetoid(root='./data/Citeseer', name='Citeseer')
dataset_Pubmed = Planetoid(root='./data/Pubmed', name='Pubmed')
# dataset_NELL = NELL(root='./data/NELL')

## 3. GAT via PyG
Implement the [Graph Attention Network](https://arxiv.org/abs/1710.10903) and reproduce their results on the datasets available via PyG.

In [190]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT_Transductive(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.head = 8
        self.feature = 8 
        self.conv1 = GATConv(in_channels = dataset.num_node_features, 
                             out_channels = self.feature, 
                             heads = self.head, 
                             dropout=0.6)
        self.conv2 = GATConv(in_channels = self.feature * self.head, 
                             out_channels = dataset.num_classes, 
                             heads=1, 
                             concat=True,
                             dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [191]:
from torch_geometric.datasets import Planetoid
dataset_cora = Planetoid(root='./data/Cora', name='Cora')

In [192]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = GAT_Transductive().to(device)
data = dataset_cora[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0: 
        print(f'Epoch : {epoch+1:03d} Loss : {loss:.4f}')

Epoch : 050 Loss : 0.3682
Epoch : 100 Loss : 0.2445
Epoch : 150 Loss : 0.3151
Epoch : 200 Loss : 0.2876


After training, we an evaluate on the test dataset:

In [193]:
model.eval() #No dropout
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8000
